# 科学技術計算 4 レポート


In [197]:
import math
import cmath
import numpy as np
import scipy
from typing import Tuple
from tqdm.auto import tqdm

rng = np.random.default_rng()

## 課題04-1

In [206]:
def lu_decomposition( A_org: np.ndarray ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A_org.ndim == 2
    assert A_org.shape[0] == A_org.shape[1]

    #前進消去を行うコード
    n = A_org.shape[0]
    A = A_org.copy()
    print("A\n", A, sep="")
    L = np.eye(n)   #単位行列生成
    P = np.eye(n)   #置換行列

    for j in range(n - 1):
        #ここから資料にあった部分ピポッド選択月も前進消去の実装
          
        #print(f"{j}-th row", A[j])
        
        # 部分ピボット選択
        j_max = np.abs(A[j:, j]).argmax() + j   #行中の一番大きい値を抽出
        if j != j_max:
            # A の行交換
            A[[j, j_max]] = A[[j_max, j]]

            # L の行交換（前の列のみ）
            if j > 0:
                L[[j, j_max], :j] = L[[j_max, j], :j]

            # P の行交換 ← これが scipy と同じ P を作るカギ
            P[[j, j_max]] = P[[j_max, j]]
        
        #print("A\n", A, sep="")
        #print("P\n", P)
    
        # 前進消去
        remaining_rows = n - (j + 1)
        r_ij = A[j + 1:, j] / A[j, j]
        A[j + 1:, j + 1:] -= np.tile(A[j, j + 1:], (remaining_rows, 1)) * r_ij.reshape(remaining_rows, -1)
        A[j + 1:, j] = 0
        L[j + 1:, j] = r_ij
        #print("A\n", A, sep="")
        #$print("L\n", L, sep="")
        
    U = A
    
    #print("U\n", U, sep="")

    P = P.T
    """
    #確認１
    print("P =",P)
    print("L =", L)
    print("U =", U)

    print("LU\n", L @ U, sep="")
    print("A\n", A_org, sep="")
    #print("LU == A", np.allclose(L @ U, A_org))
    """
    return P, L, U



for i in tqdm(range(50)):
    n = rng.integers(low = 2, high = 10) #サイズ2-10
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)
    """
    print("Psp = ", Psp)
    print("Lsp = ", Lsp)
    print("Usp = ", Usp)
    """
 
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"
    
    assert np.allclose(P, Psp), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.29680563 0.64782582 0.06735058 0.7965148  0.6989894  0.04517616
  0.16262641]
 [0.72275001 0.30019888 0.81491762 0.97193747 0.87501544 0.43603027
  0.95643269]
 [0.83500958 0.59822686 0.45962631 0.56494993 0.84323002 0.13390212
  0.05641498]
 [0.40444338 0.26188348 0.64250621 0.84390959 0.63176025 0.65713655
  0.29526265]
 [0.15734119 0.53699042 0.51654646 0.90823584 0.68556741 0.8311403
  0.19512923]
 [0.44312113 0.63120917 0.08345137 0.709508   0.52241094 0.2210318
  0.03390962]
 [0.86652824 0.91409043 0.51345203 0.26648913 0.99334332 0.83523511
  0.4230039 ]]
A
[[0.49608514 0.43946804 0.41584217 0.6230944 ]
 [0.93066544 0.49724691 0.22288528 0.88410065]
 [0.32992649 0.28174084 0.36033926 0.64779764]
 [0.47333255 0.7505196  0.20812174 0.39483049]]
A
[[0.29624784 0.09825854 0.32212249 0.21457004 0.38785649 0.60996318
  0.4535002 ]
 [0.99188588 0.89163988 0.43912232 0.690739   0.24806049 0.09107532
  0.95897498]
 [0.74769776 0.5622551  0.67030807 0.03503446 0.5203094  0.84550861


## 課題04-2


In [208]:
def solve_lu( P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray,) -> np.ndarray:
    """Solve Ax = b with PLU, LU decomposition of A = PLU

    Args:
        P (np.ndarray): nxn permutation matrix P
        L (np.ndarray): nxn lower matrix L with 1s in diagonal
        U (np.ndarray): nxn strictly upper matrix U
        b (np.ndarray): n-d rhs vector b

    Returns:
        (np.ndarray): solution x of Ax = PLUx = b
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert P.shape == (n, n)
    assert L.shape == (n, n)
    assert U.shape == (n, n)
    b = b.copy()

    # code here
    # solve Ly = P^T b
    a = P.T @ b
    # 前進代入
    y = np.zeros(n)
    for i in range(n):
        y[i] = a[i] - np.dot(L[i, :i], y[:i])
        # L[i,i] = 1（単位下三角行列）なので割り算不要

    # 後退代入
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    """
    x_true = solve(A, b)
    print("true solution x", x_true)
    print("x == x_true", np.allclose(x_true, x))
    print(x - x_true)
    """

    return x

for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    P, L, U = lu_decomposition(A)
    x_with_lu = solve_lu(P, L, U, b) #こっちが自作

    x_numpy = np.linalg.solve(A, b) #こっちがライブラリにあるやつ

    assert np.allclose(x_numpy, x_with_lu ), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.91329785 0.69869531 0.92002029 0.88951224 0.13979241 0.89614077]
 [0.29519502 0.44331084 0.51266707 0.17828152 0.31549352 0.13327151]
 [0.80765813 0.12249333 0.01151166 0.32881162 0.11188934 0.68036868]
 [0.93396004 0.2825038  0.54203057 0.11400611 0.24240487 0.49353326]
 [0.6725862  0.62746218 0.20249247 0.57839423 0.78655515 0.40937406]
 [0.42829087 0.15246454 0.31888988 0.97352879 0.44860283 0.45689621]]
A
[[0.85905588 0.88975804 0.30781038 0.21418923 0.50014906 0.76588958]
 [0.98463649 0.34303632 0.84953718 0.30802093 0.51354313 0.46290352]
 [0.92081123 0.85582966 0.40369989 0.50087495 0.66095663 0.97247608]
 [0.65796488 0.1490662  0.96826932 0.65603283 0.95066613 0.70827244]
 [0.78965966 0.45751459 0.37433238 0.90982223 0.49568833 0.52773581]
 [0.1929462  0.71037887 0.69862575 0.11790409 0.33410428 0.60240444]]
A
[[5.64621622e-01 2.70233410e-01 9.56380164e-01 4.46304590e-01
  5.83639824e-01]
 [9.08422430e-01 7.65672291e-01 6.66191195e-04 9.70182776e-02
  1.16973481e-01]
 [9.

## 課題04-4

In [ ]:
#Pの行列式計算行う関数
def det_P(A: np.ndarray) -> float:
    
    """det(P) of permutation matrix P

    Args:
        P (np.ndarray): nxn permutation matrix of 0 and 1

    Returns:
        float: det(P) that is either 1 or -1
    """

    assert P.ndim == 2
    assert P.shape[0] == P.shape[1]

    for p in P.ravel()[np.nonzero(P.ravel())]:
        assert np.isclose(p, 1.0)
    
    n = P.shape[0]
    assert np.allclose(P.sum(axis=0), np.ones(n))
    assert np.allclose(P.sum(axis=1), np.ones(n))

    ## below three lines of code are helped with ChatGPT-4o because of its complexisity
    
    # Extract the permutation using np.nonzero
    permutation = np.nonzero(P)[1]

    # Calculate the number of inversions in the permutation
    inversions = sum(1 for i in range(len(permutation)) for j in range(i + 1, len(permutation)) if permutation[i] > permutation[j])

    # Return 1 if inversions are even, otherwise -1
    return 1 if inversions % 2 == 0 else -1


#Aの行列式をPLU分解を用いて行列式の計算行う関数
def det_lu( A: np.ndarray,) -> float:
    """compute det(A)

    Args:
        A (np.ndarray): nxn square matrix

    Returns:
        (float): det(A)
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    P, L, U = lu_decomposition(A)
    P = P.T
    # code here
    #U求めちゃってかければハッピー(Lはスカラーだからたぶんどうにかなる)
    signP = det_P(P)   #Pの行列式の計算

    # ----- 2. U の対角成分の積（logで安全に計算） -----
    diagU = np.diag(U) #対角成分だけとる

    if np.any(np.isclose(diagU, 0.0)):
        return 0.0

    # 符号の計算
    signU = np.prod(np.sign(diagU))
    

    # 対角絶対値の log の和
    log_abs = np.sum(np.log(np.abs(diagU))) #logをとる

    max_log = math.log(np.finfo(float).max)    # 約 709
    min_log = math.log(np.finfo(float).tiny)   # 約 -708 (normalized min)

    if log_abs > max_log:
        # 絶対値が大きすぎて exp(log_abs) が inf になる
        abs_detU = np.inf
    elif log_abs < min_log:
        # 絶対値が小さすぎてアンダーフロー -> 実質0
        abs_detU = 0.0
    else:
        abs_detU = math.exp(log_abs)

    detU = signU * abs_detU

    #det(A) = det(P) * det(U) 
    detA = signP * detU
    return detA

#実行
for i in tqdm(range(50)):
    n = rng.integers(low=2, high=5)
    A = rng.random(size=(n, n))

    det_with_lu = det_lu(A)
    det_numpy = np.linalg.det(A)

    print(det_with_lu)
    print(det_numpy)

    assert np.isclose( det_numpy, det_with_lu), "values don't match"


  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.44898279 0.75297141]
 [0.82929658 0.22918365]]
0.5215371016177569
-0.5215371016177569


AssertionError: values don't match

## 課題04-7